# 유튜브 크롤링

본 스크립트는 2020년 9월 7일에 작성되었습니다.

## 유튜브 제목으로 URL 가져오기

Selenium과 BeautifulSoup, pandas 라이브러리를 사용해서  
  
  유튜브 제목으로 해당 영상의 url을 가져오는 스크립트다.  
  
  좋아요, 댓글, 댓글수 등 추가 데이터는 추후에 별도로 스크립트를 만들어 게재할 예정이다.

# 라이브러리 호출하기

Selenium과 BeautifulSoup, pandas 라이브러리를 호출한다.  
  
  Selenium을 사용하기 위해서는 크롬드라이버가 필수로 설치되어야 하며,  

  아래 링크를 따라 다운받으면 된다.  
[크롬드라이버 다운로드](https://chromedriver.chromium.org/downloads)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pandas as pd

import time

## 영상 제목 불러오기

검색하고자 하는 영상이 한 두개라면 그냥 검색해서 url을 검색하면 되지만,  
  
  50개 이상의 url을 가져오고자 한다면 컴퓨터의 도움을 받는 편이 낫다.
  
원하는 영상 제목을 리스트 형태로 만들어도 되지만,  

엑셀 파일을 불러온 후 그 중 하나의 컬럼을 리스트 형태로 변환해서 사용했다.

In [ ]:
yt_list = pd.read_excel('/Users/deepamine/yt_most_view_list.xlsx') #불러오고자 하는 파일의 경로와 파일명, 확장자 입력
video_list = yt_list['Video name'] #컬럼 1개를 변수로 설정
print(video_list)
url_list = [] #향후 추출한 url을 저장할 빈 리스트 생성

## Selenium을 이용한 유튜브 크롤링

주말 이틀 내내 고생해서 성공시킨 스크립트다.  

유튜브를 크롤링하는 온갖 방법을 찾아보았지만 모두 정상적으로 작동하지 않았다.  

Stockoverflow에도 나와 비슷한 고민을 가진 사람이 많다. 

아마 유튜브 측에서 주기적으로 알고리즘을 바꿔서, 크롤링이 어려운 것 같다.  

### 1) Full Xpath를 이용할 것

말 그대로 원하는 버튼의 xpath를 가져올 때, Full Xpath를 가져와야 한다.  
  
  그리고 driver.find_elements_by_xpath는 리스트 형식이므로 click 등의 selenium 명령어가 작동하지 않는다.  
  
  따라서 [0]을 붙여서 첫 번째 값을 호출해주어야 한다. (아래 코드 참조)

### 2) Selenium 작동 시 창 변환을 실시할 것

가장 고생했던 부분이다.  

주피터 노트북에서 cell을 따로 할 때는 작동이 되지만, 한 cell안에 넣거나 run all 을 수행할 때는 에러가 났다.  

반복문을 돌려야 하는 상황이기 때문에 그냥 넘어갈 수가 없었고,  

여러 시도를 한 끝에 억지로 창 변환을 하는 방식으로 해결했다.  

### 3) 크롬드라이버를 여러번 사용해야 함

한 키워드의 검색이 끝나면 크롬드라이버를 종료시키고 다시 여는 비효율적인 방법을 택했는데  

한 드라이버에서 계속 코드를 돌리면 중간에 에러가 났고 이유를 찾을 수 없었다.  

그러다 드라이버를 종료하고 다시 열어서 실행시켰더니 정상적으로 작동했고, 이것이 내가 찾은 유일한 해결방법이었다.  

혹시 더 나은 방법을 가진 분이 계시다면 댓글을 남겨주시길!


In [ ]:
chrome_driver = '/Users/deepamine/Downloads/chromedriver' #본인 pc에 설치된 chromedriver의 경로를 입력


for i in video_list:
    driver = webdriver.Chrome(chrome_driver)
    driver.get('https://www.youtube.com')

    search_bar = driver.find_elements_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div/div[1]/input')[0] #앞서 설명한 리스트 문제. 해당 xpath는 검색창의 xpath다.

    time.sleep(2) #각 액션에 앞서 2초씩 쉬어준다. 중간에 통신이 느려져 건너뛰는 문제를 방지하기 위해 2초의 간격을 두었다.
    search_bar.send_keys(i) #반복구문 (검색하고자 하는 제목 리스트를 하나씩 가져옴)
    search_bar.send_keys(Keys.RETURN) #엔터와 같은 기능
    
    time.sleep(2) 
    driver.switch_to.window(driver.window_handles[-1]) #창 변환의 기능을 하는 코드. 이게 없으면 다음 코드가 정상적으로 실행되지 않는다.

    search_name = driver.find_elements_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a/yt-formatted-string')[0]

    time.sleep(2)
    search_name.click()
    
    time.sleep(2)
    url_list.append(driver.current_url) #아까 만든 리스트에 방금 호출한 페이지의 url을 쌓는다.
    
    time.sleep(2)
    driver.close()
    
    time.sleep(2)
    
    


In [ ]:
video_list['Url'] = url_list #쌓은 url을 기존 dataframe에 추가